<a href="https://colab.research.google.com/github/Desi385/aco_for_jupyter_notebook/blob/main/TSP_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pandas
import numpy as np
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia
!pip install nltk
!pip install beautifulsoup4 vaderSentiment pandas requests matplotlib


html = requests.get('https://companiesmarketcap.com/tech/largest-tech-companies-by-market-cap/').text
soup = BeautifulSoup(html, 'html.parser')
companyName = [x.get_text(strip=True) for x in soup.find_all('div', {'class': 'company-name'})][:10]
companyCode = [x.get_text(strip=True) for x in soup.find_all('div', {'class': 'company-code'})][:10]

print(companyCode)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.3 MB/s 
['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'NVDA', 'TSM', '005930.KS', 'TCEHY', 'META']


In [ ]:
def news_headlines(ticker):
 url = f'https://finviz.com/quote.ashx?t={ticker}'
 req = requests.get(url,headers={'user-agent': 'nlp_bot'}).text
 html = BeautifulSoup(req, 'html.parser')
 headline = html.find(id='news-table')
 return headline
print(news_headlines('AAPL'))

<table border="0" cellpadding="1" cellspacing="0" class="fullview-news-outer" id="news-table" width="100%">
<tr><td align="right" width="130">Nov-01-22 12:06AM</td><td align="left"><div class="news-link-container"><div class="news-link-left"><a class="tab-link-news" href="https://finance.yahoo.com/news/1-apple-supplier-foxconn-quadruples-040647744.html" onclick="trackAndOpenNews(event, 'Reuters', 'https://finance.yahoo.com/news/1-apple-supplier-foxconn-quadruples-040647744.html');" target="_blank">UPDATE 1-Apple supplier Foxconn quadruples bonuses to staff hit by China COVID lockdown</a></div><div class="news-link-right"><span style="color:#aa6dc0;font-size:9px"> Reuters</span></div></div></td></tr>
<tr><td align="right" width="130">Oct-31-22 09:48PM</td><td align="left"><div class="news-link-container"><div class="news-link-left"><a class="tab-link-news" href="https://finance.yahoo.com/news/apple-supplier-foxconn-quadruples-bonuses-014805105.html" onclick="trackAndOpenNews(event, 'Reu

In [ ]:
class datetime_parser:
 def __init__(self):
  self.date = ''
  self.time = ''
 
 def parsing(self,dateinfo):
  dateinfo = [x.replace('\xa0\xa0', '') for x in dateinfo.split(' ')]
  if len(dateinfo) > 1:
   self.date = dateinfo[0]
   self.time = dateinfo[1]
  else:
   self.time = dateinfo[0]
  
  return datetime.strptime(self.date + " " + self.time, "%b-%d-%y %I:%M%p")

In [ ]:
def preprocessing(headline):
 dParser = datetime_parser()
 table = [x.a.text for x in headline.findAll('tr')]
 date_info = [dParser.parsing(x.td.text) for x in headline.findAll('tr')]
 display(table)
 print(date_info)

In [ ]:
def convert_to_DF(stock, news):
	stock_info = []
	for x in news:
		stock_info.append({**stock,**x})
	
	return pd.DataFrame(stock_info)


In [ ]:
def main():
	stock_info = stock_ticker_crawler(limit=3)
	news_headline_info = [preprocessing(news_headlines(x['company-code'])) for x in stock_info]
	stock_data = [convert_to_DF(x, y) for x, y in zip(stock_info, news_headline_info)]
	stock_data = pd.concat(stock_data)
	stock_data.to_excel('final_data.xlsx')
	print(stock_data)
		
if __name__ == '__main__':
	main()


NameError: ignored

In [ ]:
##Vikas_Malik Code at 06-03-2022 06:19PM
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import nltk

class datetime_parser:

	def __init__(self):
		self.date = ''
		self.time = ''
	
	def parsing(self,dateinfo):
		dateinfo = [x.replace('\xa0\xa0', '') for x in dateinfo.split(' ')]
		if len(dateinfo) > 1:
			self.date = dateinfo[0]
			self.time = dateinfo[1]
		else:
			self.time = dateinfo[0]
		
		return datetime.strptime(self.date + " " + self.time, "%b-%d-%y %I:%M%p")

def stock_ticker_crawler(limit=10):
	html = requests.get('https://companiesmarketcap.com/tech/largest-tech-companies-by-market-cap/').text
	soup = BeautifulSoup(html, 'html.parser')
	companyName = [x.get_text(strip=True) for x in soup.find_all('div', {'class': 'company-name'})][:limit]
	companyCode = [x.get_text(strip=True) for x in soup.find_all('div', {'class': 'company-code'})][:limit]
	
	return [{'company-code': x, 'company-name': y} for x, y in zip(companyCode, companyName)]	

def news_headlines(ticker):
	url = f'https://finviz.com/quote.ashx?t={ticker}'
	req = requests.get(url, headers={'user-agent': 'nlp_bot'}).text
	html = BeautifulSoup(req, 'html.parser')
	headline = html.find(id='news-table')

	return headline

def preprocessing(headline):
	dParser = datetime_parser()
	table = [x.a.text for x in headline.findAll('tr')]
	date_info = [dParser.parsing(x.td.text) for x in headline.findAll('tr')]
	
	headlines = [{"news-title": x, "date": y} for x, y in zip(table, date_info)]
	
	return headlines

def convert_to_DF(stock, news):
	stock_info = []
	for x in news:
		stock_info.append({**stock,**x})
	
	return pd.DataFrame(stock_info)

def main():
	stock_info = stock_ticker_crawler(limit=3)
	news_headline_info = [preprocessing(news_headlines(x['company-code'])) for x in stock_info]
	stock_data = [convert_to_DF(x, y) for x, y in zip(stock_info, news_headline_info)]
	stock_data = pd.concat(stock_data)
	stock_data.to_excel('final_data.xlsx')
	print(stock_data)
		
if __name__ == '__main__':
	main()

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia
def sentiment_analysis(title):
 return sia().polarity_scores(title)['compound']

In [ ]:
sia.compound



In [ ]:
help('statsforecast')

In [ ]:
pip install statsforecast


In [ ]:
from statsforecast.utils import generate_series


In [ ]:
synthetic_panel = generate_series(n_series=1, freq = 'M')


In [ ]:
synthetic_panel




In [ ]:
Y_train_df = synthetic_panel[:-6]
Y_test_df = synthetic_panel[-6:]



In [ ]:
Y_test_df

In [ ]:
from statsforecast import StatsForecast


In [ ]:
from statsforecast.models import AutoARIMA, ETS


In [ ]:
season_length = 12
horizon = len(Y_test_df)
models = [
AutoARIMA(season_length=season_length),
ETS(season_length=season_length, model='ZMZ')
]
model = StatsForecast(
df=Y_train_df,
models=models,
freq='M',
n_jobs=-1)

In [ ]:
%time
Y_hat_df = model.forecast(horizon).reset_index()
Y_hat_df.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib

fig, ax = plt.subplots(1, 1, figsize = (20, 7))
Y_hat_df = Y_test_df.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])
plot_df = pd.concat([Y_train_df, Y_hat_df]).set_index('ds')
plot_df[['y', 'AutoARIMA', 'ETS']].plot(ax=ax, linewidth=2)
ax.set_title('Forecast Synthetic Data', fontsize=22)
ax.set_ylabel('Number', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()

In [ ]:
import prophet


In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash


In [ ]:
from jupyter_core.paths import jupyter_path
import numpy as np
import pandas as pd
import seaborn as sns
import os
import glob
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import lightgbm as lgb # Our ML library



In [ ]:
# ===================== Part 2: Gradient descent =====================
print('Running Gradient Descent...')

X = np.c_[np.ones(m), X]  # Add a column of ones to X
theta = np.zeros(2)  # initialize fitting parameters

# Some gradient descent settings
iterations = 1500
alpha = 0.01

# Compute and display initial cost
print('Initial cost : ' + str(compute_cost(X, y, theta)) + ' (This value should be about 32.07)')

theta, J_history = gradient_descent(X, y, theta, alpha, iterations)

print('Theta found by gradient descent: ' + str(theta.reshape(2)))

# Plot the linear fit
plt.figure(0)
line1, = plt.plot(X[:, 1], np.dot(X, theta), label='Linear Regression')
plt.legend(handles=[line1])

input('Program paused. Press ENTER to continue')

# Predict values for population sizes of 35,000 and 70,000
predict1 = np.dot(np.array([1, 3.5]), theta)
print('For population = 35,000, we predict a profit of {:0.3f} (This value should be about 4519.77)'.format(predict1*10000))
predict2 = np.dot(np.array([1, 7]), theta)
print('For population = 70,000, we predict a profit of {:0.3f} (This value should be about 45342.45)'.format(predict2*10000))

input('Program paused. Press ENTER to continue')

# ===================== Part 3: Visualizing J(theta0, theta1) =====================
print('Visualizing J(theta0, theta1) ...')

theta0_vals = np.linspace(-10, 10, 100)
theta1_vals = np.linspace(-1, 4, 100)

xs, ys = np.meshgrid(theta0_vals, theta1_vals)
J_vals = np.zeros(xs.shape)

# Fill out J_vals
for i in range(0, theta0_vals.size):
    for j in range(0, theta1_vals.size):
        t = np.array([theta0_vals[i], theta1_vals[j]])
        J_vals[i][j] = compute_cost(X, y, t)

J_vals = np.transpose(J_vals)

fig1 = plt.figure(1)
ax = fig1.gca(projection='3d')
ax.plot_surface(xs, ys, J_vals)
plt.xlabel(r'$\theta_0$')
plt.ylabel(r'$\theta_1$')

plt.figure(2)
lvls = np.logspace(-2, 3, 20)
plt.contour(xs, ys, J_vals, levels=lvls, norm=LogNorm())
plt.plot(theta[0], theta[1], c='r', marker="x")

input('ex1 Finished. Press ENTER to exit')

In [ ]:
# Python program to reverse a linked list
# Time Complexity : O(n)
# Space Complexity : O(1)

# Node class


class Node:

	# Constructor to initialize the node object
	def __init__(self, data):
		self.data = data
		self.next = None


class LinkedList:

	# Function to initialize head
	def __init__(self):
		self.head = None

	# Function to reverse the linked list
	def reverse(self):
		prev = None
		current = self.head
		while(current is not None):
			next = current.next
			current.next = prev
			prev = current
			current = next
		self.head = prev

	# Function to insert a new node at the beginning
	def push(self, new_data):
		new_node = Node(new_data)
		new_node.next = self.head
		self.head = new_node

	# Utility function to print the linked LinkedList
	def printList(self):
		temp = self.head
		while(temp):
			print(temp.data, end=" ")
			temp = temp.next


# Driver code
llist = LinkedList()
llist.push(20)
llist.push(4)
llist.push(15)
llist.push(85)

print ("Given Linked List")
llist.printList()
llist.reverse()
print ("\nReversed Linked List")
llist.printList()

# This code is contributed by Vikas Malik (desi385)


In [ ]:
import pandas as pd
import numpy as np
!pip install haversine
import math
from haversine import haversine, Unit
import random, sys, time, math
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
import seaborn as sb
warnings.filterwarnings('ignore')
plt.style.use('seaborn-whitegrid')




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
    class ACO():
      def __init__(self):
        self.y = []
        self.z = []

    def calculate_distance(self, paths):
        total_distance = 0;
        x = paths[0]
        for p in range(len(paths)): 
            i = paths[p]
            total_distance += _distance[x][i]
            if verbose:
                print(x, i, _distance[x][i], total_distance)
            x = i

        if verbose:
            print(total_distance)
        return total_distance
        print(total_distance)
    def Fit(self, distance, nk=10, maxIterations=100, beta=2, zeta=0.1, rho=0.2, q0=0.7, plot=False, verbose=False):
        _distance = distance 
        X = distance[0]
        Y = X
        
        # initial information Transfer
        total_distance = 0;
        x = 0
        for i in range(len(X)):    
            total_distance += _distance[x][i]
            x = i

        _Info_transfer_InitialValue = 1/total_distance
        if verbose:
            print(total_distance, _Info_transfer_InitialValue)
            print()

        _Info_transfer_ = []        
        for x in X:      
            result = [_Info_transfer_InitialValue for y in Y]
            _Info_transfer_.append(result)

        if depot == -1: # start from random node
            _depot = random.sample(range(0, len(df1)), nk)
        else:
            _depot = [depot] * nk 

        if verbose:
            print('_depot')
            print(_depot)
            print()

        
        best_paths = []
        best_distance = sys.float_info.max

        start_time_process = time.time()


        for it in range(maxIterations):
            best_current_paths = []

            best_current_distance = sys.float_info.max
            total_node = 1
            exist = []
            for ant in range(nk):
                if verbose:
                    print()
                    print("Semut", ant, " ... ", _depot[ant])

                paths = []
                exist = [] 
                _from = _depot[ant]
                paths.append(_depot[ant])
                exist.append(_depot[ant])


                while len(paths) < len(df1):    
                    # get all node not visited yet
                    _posibleNode = []
                    for i in range(len(X)):
                        _exist = False
                        for x in exist:
                            if i == x:
                                _exist = True
                        if not _exist:
                            _posibleNode.append(i)     
                    _q0 = np.random.randn()
                    if _q0 <= q0:
                        # get by heuristic 
                        if verbose:
                            print("heuristic")

                        shortTransition = sys.float_info.max
                        __to = -1
                        for p in _posibleNode:
                            distanceValue = _distance[_from][p]
                            Info_transfer_Value = _Info_transfer_[_from][p]
                            transitionValue = (1/Info_transfer_Value) * math.pow(distanceValue, beta)
                            if verbose:
                                print(_from, p, distanceValue, Info_transfer_Value, transitionValue)
                            if shortTransition > transitionValue:
                                shortTransition = transitionValue
                                _to = p

                        paths.append(_to)
                        exist.append(_to)
                        if verbose:
                            print('next: ', _to)  
                    else:    
                        # get by random
                        if verbose:
                            print("random")
                        _to = random.randint(0, len(df)) 
                        while _to in exist:
                            _to = random.randint(0, len(df)) 
                        paths.append(_to)
                        exist.append(_to)

                        if verbose:
                            print('next: ', _to)
                    # Apply local Info_transfer_ updating
                    Info_transfer_Value = _Info_transfer_[_from][_to]
                    tau = (( 1 - zeta) * Info_transfer_Value) + _Info_transfer_InitialValue
                    _Info_transfer_[_from][_to] = tau
                    _Info_transfer_[_to][_from] = tau

                    _from = _to

                    if verbose:
                        print(paths)
                        print()

                distance = self.calculate_distance(paths)

                if best_current_distance > distance:
                    best_current_distance = distance 
                    best_current_paths = paths



            if best_distance > best_current_distance:
                best_distance = best_current_distance
                best_paths = best_current_paths

            self.y.append(best_distance)
            self.z.append(best_current_distance)
        # Apply global Info_transfer_ updating
            __from = paths[0]
            for nn in range(len(paths)):
                __to = paths[nn]
                if __from != __to:
                    Info_transfer_Value = _Info_transfer_[__from][__to]
                    tau = (( 1 - rho) * Info_transfer_Value) + _Info_transfer_InitialValue
                    _Info_transfer_[__from][__to] = tau
                    _Info_transfer_[__to][__from] = tau
                __from = __to


            print(it, " ... ", distance, ":", best_distance)

        print()
        print("All Process --- %s seconds ---" % (time.time() - start_time_process))

        #     print(best_current_paths)
        print('Best path')
        print(best_paths)
        result = self.calculate_distance(best_paths)
        print('Best distance')
        print(result)
        print()           
        return best_paths, result

In [ ]:
def Plot(self):
        fig = plt.figure()
        ax = plt.axes()
        y = np.array(self.y)
        z = np.array(self.z)
        __w = np.average(self.z)
        w = [__w] * maxIterations
        w = np.array(w)
        x = np.linspace(0, maxIterations, maxIterations)
        ax.plot(x, y, z);
        ax.plot(x, w, z);

In [ ]:
!git clone https://github.com/tribasuki74/aco_for_jupyter_notebook

Cloning into 'aco_for_jupyter_notebook'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 12 (delta 0), reused 2 (delta 0), pack-reused 9
Unpacking objects: 100% (12/12), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_url = 'https://raw.githubusercontent.com/Desi385/aco_for_jupyter_notebook/main/dataset/vdata_main.csv'
_file = file_url.strip(".csv")


Mounted at /content/drive


In [ ]:
file_result =  '/content/drive/My Drive/Val_5k_final' + '_Result_ACO.csv'
file_excel = '/content/drive/My Drive/Vikas' + '_DF.xlsx'

df = pd.read_csv(file_url)

In [ ]:

#file_url = 'https://raw.githubusercontent.com/Desi385/aco_for_jupyter_notebook/main/dataset/data_main.csv'#
#_file = file_url.strip(".csv")
#file_result =  _file + 'https://raw.githubusercontent.com/Desi385/aco_for_jupyter_notebook/main/dataset/data_Result_ACO.csv'
#df = pd.read_csv(file_url)


In [ ]:
_job = df.groupby('cluster')['tracking_id'].count().sort_values(ascending=[False])
print(_job)
# print(len(_job))
print()

max_capacity = 0
for nc in range(len(_job)):
       if _job[nc] > max_capacity:
        max_capacity = _job[nc]
        
df1 = df[['lat', 'lng']]
print(len(df1), " data processing ... ") 
print()

depot_lat = 2.971718
depot_lng = 101.608376
depot = 0

_data = df[['lat', 'lng']]    
_data.loc[-1] = [depot_lng, depot_lat]  # adding a row
_data.index = _data.index + 1  # shifting index
_data.sort_index(inplace=True) 

df1 = _data[['lng', 'lat']]
problem = np.array(df1)
distance = []
for i in range(len(_data)):
    lat = _data.iloc[i]['lat']
    lng = _data.iloc[i]['lng']
    from_node = (lng, lat)
    result = []
    for j in range(len(_data)):
        lat = _data.iloc[j]['lat']
        lng = _data.iloc[j]['lng']
        to_node = (lng, lat)
        if i == 0:
            dist = 0
        elif j == 0:
            dist = 0
        else:
            dist = round(haversine(from_node, to_node, unit=Unit.METERS) * 2)
        result.append(dist/1000)
    distance.append(result)
    
#print(distance) 
#print()
df5 = pd.DataFrame(distance)
df5

cluster
0    1000
2    1000
3    1000
4    1000
1     999
5     564
Name: tracking_id, dtype: int64

5563  data processing ... 



In [ ]:

nk = 5  # number of k = ants
maxIterations = 100
beta =  4  # Heuristic constant
zeta = 0.4 # local Pheromone decay   
rho = 0.2  # global Pheromone decay
q0 = 0.7   # randomnize
plot = False
verbose = False

optimizer = ACO()
best_paths, result = optimizer.Fit(distance, nk, maxIterations, beta, zeta, rho, q0, plot, verbose)

optimizer.Plot()

# convert into list New order
# -------
_newCluster = [0] * len(df)
_newOrder = [0] * len(df)

_order = 1
for o in range(len(best_paths)):
    if o > 0:
        pos = best_paths[o]

        _newCluster[pos-1] = 0
        _newOrder[pos-1] = _order
        if verbose:
            print(o, pos)
            print(_newCluster)
            print(_newOrder)

        _order += 1

print(_newCluster)
print(_newOrder)
df['New Cluster'] = _newCluster
df['New Order'] = _newOrder
print("Result was saving ... ", file_result)
df.to_csv(file_result, index=False)  
